In [1]:
import requests
import pandas as pd
import json
from dotenv import load_dotenv
import os
from datetime import datetime
import re
from sqlalchemy import create_engine
from zoneinfo import ZoneInfo

In [2]:
url = "	https://data.taipei/api/dataset/63f31c7e-7fc3-418b-bd82-b95158755b4d/resource/eb481f58-1238-4cff-8caa-fa7bb20cb4f4/download"

In [3]:
response = requests.get(url=url)

In [4]:
response.text

'\ufeffSeqNo,年月,URL\r\n1,201701,http://tcgmetro.blob.core.windows.net/stationod/%E8%87%BA%E5%8C%97%E6%8D%B7%E9%81%8B%E6%AF%8F%E6%97%A5%E5%88%86%E6%99%82%E5%90%84%E7%AB%99OD%E6%B5%81%E9%87%8F%E7%B5%B1%E8%A8%88%E8%B3%87%E6%96%99_201701.csv\r\n2,201702,http://tcgmetro.blob.core.windows.net/stationod/%E8%87%BA%E5%8C%97%E6%8D%B7%E9%81%8B%E6%AF%8F%E6%97%A5%E5%88%86%E6%99%82%E5%90%84%E7%AB%99OD%E6%B5%81%E9%87%8F%E7%B5%B1%E8%A8%88%E8%B3%87%E6%96%99_201702.csv\r\n3,201703,http://tcgmetro.blob.core.windows.net/stationod/%E8%87%BA%E5%8C%97%E6%8D%B7%E9%81%8B%E6%AF%8F%E6%97%A5%E5%88%86%E6%99%82%E5%90%84%E7%AB%99OD%E6%B5%81%E9%87%8F%E7%B5%B1%E8%A8%88%E8%B3%87%E6%96%99_201703.csv\r\n4,201704,http://tcgmetro.blob.core.windows.net/stationod/%E8%87%BA%E5%8C%97%E6%8D%B7%E9%81%8B%E6%AF%8F%E6%97%A5%E5%88%86%E6%99%82%E5%90%84%E7%AB%99OD%E6%B5%81%E9%87%8F%E7%B5%B1%E8%A8%88%E8%B3%87%E6%96%99_201704.csv\r\n5,201705,http://tcgmetro.blob.core.windows.net/stationod/%E8%87%BA%E5%8C%97%E6%8D%B7%E9%81%8B%E6%AF%8F%E6

In [10]:
response.text.split("\r")

['\ufeffSeqNo,年月,URL',
 '\n1,201701,http://tcgmetro.blob.core.windows.net/stationod/%E8%87%BA%E5%8C%97%E6%8D%B7%E9%81%8B%E6%AF%8F%E6%97%A5%E5%88%86%E6%99%82%E5%90%84%E7%AB%99OD%E6%B5%81%E9%87%8F%E7%B5%B1%E8%A8%88%E8%B3%87%E6%96%99_201701.csv',
 '\n2,201702,http://tcgmetro.blob.core.windows.net/stationod/%E8%87%BA%E5%8C%97%E6%8D%B7%E9%81%8B%E6%AF%8F%E6%97%A5%E5%88%86%E6%99%82%E5%90%84%E7%AB%99OD%E6%B5%81%E9%87%8F%E7%B5%B1%E8%A8%88%E8%B3%87%E6%96%99_201702.csv',
 '\n3,201703,http://tcgmetro.blob.core.windows.net/stationod/%E8%87%BA%E5%8C%97%E6%8D%B7%E9%81%8B%E6%AF%8F%E6%97%A5%E5%88%86%E6%99%82%E5%90%84%E7%AB%99OD%E6%B5%81%E9%87%8F%E7%B5%B1%E8%A8%88%E8%B3%87%E6%96%99_201703.csv',
 '\n4,201704,http://tcgmetro.blob.core.windows.net/stationod/%E8%87%BA%E5%8C%97%E6%8D%B7%E9%81%8B%E6%AF%8F%E6%97%A5%E5%88%86%E6%99%82%E5%90%84%E7%AB%99OD%E6%B5%81%E9%87%8F%E7%B5%B1%E8%A8%88%E8%B3%87%E6%96%99_201704.csv',
 '\n5,201705,http://tcgmetro.blob.core.windows.net/stationod/%E8%87%BA%E5%8C%97%E6%8D%B7%E9%8

In [17]:
response_list = response.text.split("\r")
col_name = response_list[0].split(",")
df = pd.DataFrame(columns=col_name[1:])
df


,年月,URL


In [31]:
url_df = pd.concat([pd.DataFrame([response_list[i].split(",")[1:]],columns=col_name[1:]) for i in range(1,len(response_list))],axis=0)
url_df.reset_index(drop=True,inplace=True)
url_df

,年月,URL
0,201701,http://tcgmetro.blob.core.windows.net/stationo...
1,201702,http://tcgmetro.blob.core.windows.net/stationo...
2,201703,http://tcgmetro.blob.core.windows.net/stationo...
3,201704,http://tcgmetro.blob.core.windows.net/stationo...
4,201705,http://tcgmetro.blob.core.windows.net/stationo...
...,...,...
82,202311,http://tcgmetro.blob.core.windows.net/stationo...
83,202312,http://tcgmetro.blob.core.windows.net/stationo...
84,202401,http://tcgmetro.blob.core.windows.net/stationo...
85,202402,http://tcgmetro.blob.core.windows.net/stationo...


In [34]:
url = url_df.loc[1,["URL"]]
response = requests.get(url=url)

URL    http://tcgmetro.blob.core.windows.net/stationo...
Name: 1, dtype: object

In [11]:
from io import StringIO

In [13]:
pd.read_csv(StringIO(response.text))

,SeqNo,年月,URL
0,1,201701,http://tcgmetro.blob.core.windows.net/stationo...
1,2,201702,http://tcgmetro.blob.core.windows.net/stationo...
2,3,201703,http://tcgmetro.blob.core.windows.net/stationo...
3,4,201704,http://tcgmetro.blob.core.windows.net/stationo...
4,5,201705,http://tcgmetro.blob.core.windows.net/stationo...
...,...,...,...
82,83,202311,http://tcgmetro.blob.core.windows.net/stationo...
83,84,202312,http://tcgmetro.blob.core.windows.net/stationo...
84,85,202401,http://tcgmetro.blob.core.windows.net/stationo...
85,86,202402,http://tcgmetro.blob.core.windows.net/stationo...
